In [1]:
%load_ext cython

# Specifying C data types in Cython

In [2]:
%%cython

def f(): # nonsense function
    cdef unsigned wonder_of_the_world = 7
    cdef int age = 21  # years
    cdef float height = 1.83 # metres
    cdef long long total_debt = 18000000000000  # 18 trillion
    cdef double milky_way_mass = 6e42  # kg
    cdef bint valid = False # boolean in cython
    return wonder_of_the_world + age + height + total_debt + milky_way_mass 

In [3]:
f()

6e+42

# Cython functions

In [4]:
%%cython

# pay attention to the differences of def, cdef, cpdef functions

# a normal python function (can be called in python and cython)
def f1(a, b):
    return a + b

# a normal python function with data types (can be called in python and cython)
# can't specify return type, returns always a python object
def f2(int a, int b):
    return a + b

# can be called only in cython (--> fast)
cdef int f3(int a, int b):
    return a + b

# can be called in cython and python
cpdef int f4(int a, int b):
    return f3(a, b)

In [5]:
# call functions in python
print(f1(1, 2))
print(f2(1, 2))
print(f4(1, 2))

3
3
3


# Docstring plus call signature

In [6]:
%%cython
# cython: embedsignature=True

def my_add(int a, int b):
    """ This function adds two integer. """
    return a + b

In [7]:
help(my_add)

Help on built-in function my_add in module _cython_magic_506ff99843adbb2d83d62c471e6e8278:

my_add(...)
    my_add(int a, int b)
    This function adds two integer.



# Declaring built-in Python types
May speed up execution and is good for type safety. 

- **data structure types**: list, tuple, dict, set, frozenset
- **string types**: str, unicode, bytes, bytearray
- **several other data types**: complex, array, date, time, datetime, ...

In [8]:
%%cython

def h(x, int n):
    cdef int i
    for i in range(n):
        x.add(i)
    return x

def k(set x, int n): # x is a set, other types are invalid (type safety)
    cdef int i
    for i in range(n):
        x.add(i)
    return x

In [9]:
%timeit h(set(), 1000000)
%timeit k(set(), 1000000)

99 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
65.5 ms ± 499 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# C-level data structures (array, struct, enum, union)

In [10]:
%%cython

cdef enum Rating:
    BAD = 0
    GOOD = 1

cdef struct book: 
    int isbn
    Rating rating
    
def getbook():
    cdef book x
    x.isbn = 12345
    x.rating = GOOD
    return x

def getbooks():
    cdef book books[2] # This is a static array
    books[0] = getbook()
    
    books[1].isbn = 56789
    books[1].rating = BAD
    return books

In [11]:
print(getbook()) # cython added code to convert struct automatically to a python dictionary
print(getbooks()) # cython added code to convert array automatically to a python list

{'isbn': 12345, 'rating': 1}
[{'isbn': 12345, 'rating': 1}, {'isbn': 56789, 'rating': 0}]


In [12]:
%%cython
# cython: language_level=3

cdef union transformer:
    # these variables use the same memory block
    long long x  # 8 bytes
    double d     # 8 bytes also
    
def test():
    cdef transformer u
    u.d = 0.0
    print('u.d Before: ', u.x)    
    u.d = 1.0
    print('u.d After : ', u.x)

In [13]:
test()

u.d Before:  0
u.d After :  4607182418800017408


# Typed memoryviews
Memory views are useful for passing Python or Numpy arrays to Cython.

In [14]:
%%cython

# data type alias
ctypedef unsigned long long uint64 

# view is a matrix
def fill_square(uint64[:,:] view, uint64 num):
    view[-2:,-2:] = num # slicing in two dimensions
    

# slightly faster version of fill_square
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def fill_square_loop(uint64[:,:] view, uint64 num): 
    cdef:
        int i, j
        int rows = view.shape[0] # amount rows
        int columns = view.shape[1] # amount columns
    for i in range(rows - 2, rows):
        for j in range(columns - 2, columns):
            view[i, j] = num
        

In [15]:
import numpy
x = numpy.zeros(shape=(4, 4), dtype=numpy.uint64)
%timeit fill_square(x, 1)
print(x)
%timeit fill_square_loop(x, 2)
print(x) 

671 ns ± 9.36 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 1 1]]
611 ns ± 3.75 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
[[0 0 0 0]
 [0 0 0 0]
 [0 0 2 2]
 [0 0 2 2]]


# Passing Python strings, working with C strings

In [16]:
%%cython

# in python 3 it is not allowed to assign python strings directly to C strings
def f(str text):
    cdef char *s 
    bstr = text.encode('utf-8') # we must encode to bytes the string
    s = bstr
    print(s)
    
def fb(bytes text): # passing a bytes object directly
    cdef char *s = text
    print(s)  
    
# just use python strings inside cython (fast enough)
def fs(str text):
    print(text)

In [17]:
f("cat")
fb(b"dog")
fs("mini pig")

b'cat'
b'dog'
mini pig


# Passing a pointer into a function

In [18]:
%%cython
# cython: language_level=3

cdef f(int* px):
    px[0] += 1 # de-reference with array access
    
def g():
    cdef int x = 0 
    cdef int* px =  &x
    print('Before calling f: x is', x)
    f(px)
    print('After calling f: x is', x)

In [19]:
g()

Before calling f: x is 0
After calling f: x is 1


# Type-casting

In [20]:
%%cython

def h():
    x = ['a', 'b', 'c']
    
    # casting in cython is done with angle brackets
    cdef void* p = <void*>x  # get a void pointer to the python object
    
    # the memory address is the same as the object id()
    print(<long>p)
    print(id(x))
    
    # by type-casting to <object>, you can get the original python object back
    print(<object>p)

In [21]:
h()

140684758768456
140684758768456
['a', 'b', 'c']
